In [169]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import csv
import json
import re

In [170]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»','&nbsp':' ',u'\xa0': ' ',
                '\n':' ','\t':' ','    ':'','&emsp':' ',
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        #str = str.replace(k[1:], v)
    str = str.strip('\n')
    str = str.strip(' ')

    return str

In [171]:
def SingleHTMLProcess(path):

    dic={"Proposal Number":"", "Phase 1 Contract #":"","Subtopic Title":"","Proposal Title":"","Small Business Concern_Firm":"","Small Business Concern_Address":"","Small Business Concern_Phone":"",
         "Principal Investigator_Name":"","Principal Investigator_E-mail":"","Principal Investigator_Address":"","Principal Investigator_Phone":"",
         "Business Official_Name":"","Business Official_E-mail":"","Business Official_Address":"","Business Official_Phone":"",
         "TRL_Begin":"","TRL_End":"","Technical Abstract":"","Potential NASA applications":"","Potential non-NASA applications":"",
         "Technology Taxonomy Mapping":""}

    #print(len(dic.keys()))

    htmlfile = open(path, 'r', encoding='utf-8')
    html=htmlfile  #(htmlfile.replace('<br>','')).replace('<br/>','').read()
    #html=html.replace('<br>','')
    bs = BeautifulSoup(html, "html.parser")  # 缩进格式

    tds = bs.find_all("td")   #info

    for i in range(len(tds)):
        #str = divs1[i].string
        if (tds[i] is not None):
            str = tds[i].get_text()
            if (str.find("PROPOSAL NUMBER") != -1):
                # Proposal number
                str=filterHTMLstr(tds[i+1].get_text())
                str="01-1 "+str
                dic["Proposal Number"]=str
            elif (str.find("PHASE-I CONTRACT NUMBER") != -1):
                # Phase 1 Contract Number
                dic["Phase 1 Contract #"] = filterHTMLstr(tds[i+1].get_text())
            elif (str.find("SUBTOPIC TITLE") != -1):
                # Subtopic Title
                dic["Subtopic Title"] = filterHTMLstr(tds[i+1].get_text())
            elif (str.find("PROPOSAL TITLE") != -1):
                # Proposal Title
                dic["Proposal Title"] = filterHTMLstr(tds[i+1].get_text())

    ### technical abstract
    htmlfile = open(path, 'r', encoding='utf-8')
    html=htmlfile  #(htmlfile.replace('<br>','')).replace('<br/>','').read()
    bs = BeautifulSoup(html, "lxml")  # 缩进格式
    technicalAbstract = ""
#     print(path)
    for sibling in bs.find_all('p')[0].find_all('b')[0].next_siblings:
        if sibling.get_text().strip() != "":
            technicalAbstract += sibling.get_text()
    
    ### potential commercial application
    commercialAbstract = ""
    for sibling in bs.find_all('p')[1].find_all('b')[0].next_siblings:
        if sibling.get_text().strip() != "":
            commercialAbstract += sibling.get_text()
            
    ### Name and Address of Principal Investigator
    investigatorInfos = []
    for sibling in bs.find_all('p')[2].find_all('font')[0].next_siblings:
        if sibling.get_text().strip() != "":
            investigatorInfos.append(filterHTMLstr(sibling.get_text()))
    investigator_Name = investigatorInfos[0]
    investigator_Addr = investigatorInfos[2].rstrip() + ", " + investigatorInfos[3]
    
    ### Small business concern firm and address
    smbInfos = []
    for sibling in bs.find_all('p')[3].find_all('font')[0].next_siblings:
        if sibling.get_text().strip() != "":
            smbInfos.append(filterHTMLstr(sibling.get_text()))
    smb_Name = smbInfos[0]
    smb_Addr = smbInfos[1].rstrip() + ", " + smbInfos[2]
        
    dic["Technical Abstract"] = technicalAbstract
    dic["Potential non-NASA applications"] = commercialAbstract
    dic["Principal Investigator_Name"] = investigator_Name
    dic["Principal Investigator_Address"] = investigator_Addr
    dic["Small Business Concern_Firm"] = smb_Name
    dic["Small Business Concern_Address"] = smb_Addr
    return dic


In [90]:
def ReadFiles(Directory_path):     # Read all the html files
    path=Directory_path   # The directory
    file_names = os.listdir(path)  # 得到文件夹下的所有文件名称
    files_position=[]
    for file_name in file_names:
        position=path+"/"+file_name
        #print(position)
        files_position.append(position)

    print("Total number of HTML files: ", len(files_position))
    return files_position

def MultipleFileProcess(files_position):
    totaldata = []
    for position in files_position:
        data = SingleHTMLProcess(position)
        totaldata.append(data)

    return totaldata

def to_Excel(totaldata):
    output = open('data21_sbir_phase2.xls', 'w', encoding='utf-8')
    # the title of each column
    output.write('Proposal Number\tPhase 1 Contract #\tSubtopic Title\tProposal Title\tSmall Business Concern_Firm\tSmall Business Concern_Address\tSmall Business Concern_Phone\t'
                 'Principal Investigator_Name\tPrincipal Investigator_E-mail\tPrincipal Investigator_Address\tPrincipal Investigator_Phone\t'
                 'Business Official_Name\tBusiness Official_E-mail\tBusiness Official_Address\tBusiness Official_Phone\t'
                 'TRL_Begin\tTRL_End\tTechnical Abstract\tPotential NASA applications\tPotential non-NASA applications\n')

    for i in range(len(totaldata)):
        for j in range(len(totaldata[i])):
            output.write(str(totaldata[i][j]))  # write函数不能写int类型的参数，所以使用str()转化
            output.write('\t')  # 相当于Tab一下，换一个单元格
        output.write('\n')  # 写完一行立马换行
    output.close()

def to_CSV(totaldata):
    pd.DataFrame(totaldata).to_csv('./output/01_SBIR_phase1.csv',index=False, encoding="utf-8")

def to_json(totaldata):
    json_str = json.dumps(totaldata)
    with open('../test_data.json', 'w') as json_file:
        json_file.write(json_str)

In [168]:
Directory_path = "../Datasets/01/sbir/phase1"
files_position = ReadFiles(Directory_path)
totaldata=MultipleFileProcess(files_position)
to_CSV(totaldata)

Total number of HTML files:  306


In [ ]:
pandas

### Test Module

In [158]:
Directory_path = "../Datasets/01/sbir/phase1"
files_position = ReadFiles(Directory_path)
# path = files_position[0]
path = '../Datasets/01/sbir/phase1/SBIR-01-1-B2.01-9305.html'
htmlfile = open(path, 'r', encoding='utf-8')
html = htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式

Total number of HTML files:  306


In [159]:
bs.find_all('p')[1]

<p>
<b>POTENTIAL COMMERCIAL APPLICATIONS</b><br/>
With the expected high selectivity and low detection limit and versatility of the measurement procedure, our proposed program will result in simple low cost devices that will work independently of gravity and that can be easily miniaturized for the different applications. In addition to NASA?s gas sensing needs, such sensors can be of benefit to the automotive sector for detection of pollutant gases emanating from automobile exhaust.
</p>

In [160]:
### technical abstract
technicalAbstract = ""
for sibling in bs.find_all('p')[0].find_all('b')[0].next_siblings:
    if sibling.get_text().strip() != "":
        technicalAbstract += sibling.get_text()
print(filterHTMLstr(technicalAbstract))

Gas sensors based on metal oxide sensing layers, are known to be widely used for the measurement of CO, CO2, aromatic hydrocarbon, NO2 etc., but such sensors lack selectivity in general, and operation at high temperatures is required. Arrays of sensors distinguished by their cross sensitivity have to be used where a certain degree of selectivity is obtained. The proposed program will focus on the use of highly organized ultrathin films of conducting polymers and their nanocomposites with high conductivity, which will selectively recognize the above gases with high sensitivity (#ppb). In Phase I, we will synthesize several high conductivity polymers which we will process using the supramolecular approach via the layer-by-layer self-assembly technique to fabricate the necessary films for gas sensing applications for the first time with emphasis on sensitivity and selectivity that are otherwise unattainable with other fabrication methods or materials. Regioregular polythiophene derivative

In [161]:
### potential commercial application
commercialAbstract = ""
for sibling in bs.find_all('p')[1].find_all('b')[0].next_siblings:
    if sibling.get_text().strip() != "":
        commercialAbstract += sibling.get_text()
print(filterHTMLstr(commercialAbstract))

With the expected high selectivity and low detection limit and versatility of the measurement procedure, our proposed program will result in simple low cost devices that will work independently of gravity and that can be easily miniaturized for the different applications. In addition to NASA?s gas sensing needs, such sensors can be of benefit to the automotive sector for detection of pollutant gases emanating from automobile exhaust.


In [162]:
### Name and Address of Principal Investigator
print(bs.find_all('p')[2])
investigatorInfos = []
for sibling in bs.find_all('p')[2].find_all('font')[0].next_siblings:
    if sibling.get_text().strip() != "":
        investigatorInfos.append(filterHTMLstr(sibling.get_text()))
print(investigatorInfos)
investigator_Name = investigatorInfos[0]
investigator_Addr = investigatorInfos[2].rstrip() + ", " + investigatorInfos[3]
print(investigator_Name)
print(investigator_Addr)

<p>
<b>NAME AND ADDRESS OF PRINCIPAL INVESTIGATOR</b>
<font size="-1">(Name, Organization Name, Mail Address, City/State/Zip)</font><br/>
Jimmy Krozel, Ph.D. <br/>
Metron, Inc. <br/>
131 Elden Street, Suite 200 <br/>
Herndon , VA    20170 - 4835 <br/>
</p>
['Jimmy Krozel, Ph.D.', 'Metron, Inc.', '131 Elden Street, Suite 200', 'Herndon , VA20170 - 4835']
Jimmy Krozel, Ph.D.
131 Elden Street, Suite 200, Herndon , VA20170 - 4835


In [88]:
### Small business concern firm and address
smbInfos = []
for sibling in bs.find_all('p')[3].find_all('font')[0].next_siblings:
    if sibling.get_text().strip() != "":
        smbInfos.append(filterHTMLstr(sibling.get_text()))
smb_Name = smbInfos[0]
smb_Addr = smbInfos[1].rstrip() + ", " + smbInfos[2]
print(smb_Name)
print(smb_Addr)

Global Aerospace Corporation
711 West Woodbury Road, Suite H, Altadena , CA   91001 - 5327
